# Tutorial Convert Solvency 2 XBRL-instances to CSV, HTML and pickles

This tutorial describes how to convert XBRL-instances to csv-, html- and pickle files per template.

We use Arelle, a open source package for processing XBRL. In addition this repository contains code to process the Solvency 2 and FTK instances efficiently.

In [1]:
from arelle import ModelManager, Cntlr, ModelXbrl, XbrlConst, RenderingEvaluator, \
                   ViewFileRenderedGrid, ModelFormulaObject

In [2]:
import src
import pandas as pd
from os import listdir, walk, makedirs, environ
from os.path import isfile, join, exists, basename
from datetime import datetime

## Initialize the Arelle model manager

First we specify the directories with the taxonomy and instances. You can put your own instances in the data/instances directory, or you can specify here the directories that you want to use.

In [3]:
# the taxonomy should be data/taxonomy/arelle
# the instances you want to use should be in data/instances

XBRL_TAXONOMY_PATH = join('..', 'data', 'taxonomy')
XBRL_INSTANCES_PATH = join('..', 'data', 'instances')

LANGUAGE = "en-GB"
environ['XDG_CONFIG_HOME'] = XBRL_TAXONOMY_PATH

# The role defined in the model.xsd schema for resources representing codes of rows or columns is
euRCcode = 'http://www.eurofiling.info/xbrl/role/rc-code'

To process XBRL, we need a controller and a modelmanager object. 

In the controller you can specify logging. Here we have set logging to print in this notebook.

In [4]:
# Now we make a modelmanager
# logFileName = "logToPrint" -> logging is print to notebook
# logFileName = "arelle.log" -> logging is to filename (use .json or .xml for specific format)

controller = Cntlr.Cntlr(logFileName = "logToPrint")
controller.webCache.workOffline = True
controller.logger.messageCodeFilter = None

modelmanager = ModelManager.initialize(controller)
modelmanager.defaultLang = LANGUAGE
modelmanager.formulaOptions = ModelFormulaObject.FormulaOptions()
modelmanager.loadCustomTransforms()

## Read XBRL-instance in the modelmanager

Now we are able to read and process an XBRL-instance.

We read the example instances provided with the taxonomy.

In [6]:
# the example instance of the quarterly templates for solo
instance_name = 'qrs_240_instance.xbrl'  

# the example instance of the annual templates
# instance_name = 'aeb_240_instance.xbrl'

# the example instance of the FTK assets templates
instance_name = 'DNB-NR_FTK-2019-06_2019-12-31_MOD_FTK-BEL.XBRL'

In [7]:
xbrl_instance = ModelXbrl.load(modelmanager, join(XBRL_INSTANCES_PATH, instance_name))
RenderingEvaluator.init(xbrl_instance)

2020-09-09 15:48:05,867 [] Formula xpath2 grammar initialized in 2.87 secs - 

2020-09-09 15:48:09,743 [info:profileActivity] ... formula parameter checks 3.864 secs
 - DNB-NR_FTK-2019-06_2019-12-31_MOD_FTK-BEL.XBRL 

2020-09-09 15:48:12,028 [info:profileActivity] ... custom function checks and compilation 2.285 secs
 - DNB-NR_FTK-2019-06_2019-12-31_MOD_FTK-BEL.XBRL 

2020-09-09 15:48:14,453 [info:profileActivity] ... parameter checks and select evaluation 2.424 secs
 - DNB-NR_FTK-2019-06_2019-12-31_MOD_FTK-BEL.XBRL 



## Convert XBRL-instance to CSV and Pandas-pickle

For each template or table in the instance we export the results to a csv file and a Pandas pickle-file. 

A Pandas pickle-file maintains the correct indices, whereas the csv does not, so if you want to access the data read the pickle (we included an example below).

The csv-files and the pickle-files are stored in a subdirectory identical to the name of the XBRL-instance (without extension)

In [8]:
# The location of the csv-files
subdir = basename(instance_name).split(".")[0]

In [9]:
# get tables in instance and sort by short name and print the first ten tables
tables = list(xbrl_instance.modelRenderingTables)
tables.sort(key = lambda table: table.genLabel(lang = LANGUAGE,strip = True, role = euRCcode))
for table in tables[0:10]:
    print(table.genLabel(lang = LANGUAGE,strip = True, role = euRCcode))

K000
K050
K101-1
K101-2
K201-1
K201-2
K202
K203
K204
K205


In [10]:
# create csv and pickle files
# time_stamp = datetime.now().strftime("%Y_%m_%d-%H_%M_%S")

# use verbose_labels = False if you want the row-column code as column names
# use verbose_labels = True if you want labels as column names

for table in tables:
    obj = src.generateCSV.generateCSVTables(xbrl_instance, join(XBRL_INSTANCES_PATH, subdir), 
                                            table = table, 
                                            lang = LANGUAGE,
                                            verbose_labels = False)

2020-09-09 15:48:17,092 []  ... directory ..\data\instances\DNB-NR_FTK-2019-06_2019-12-31_MOD_FTK-BEL created - 

2020-09-09 15:48:17,093 []  ... saved output ..\data\instances\DNB-NR_FTK-2019-06_2019-12-31_MOD_FTK-BEL\FTK.K000.csv and .pickle - 

2020-09-09 15:48:17,667 []  ... saved output ..\data\instances\DNB-NR_FTK-2019-06_2019-12-31_MOD_FTK-BEL\FTK.K050.csv and .pickle - 

2020-09-09 15:48:18,515 []  ... saved output ..\data\instances\DNB-NR_FTK-2019-06_2019-12-31_MOD_FTK-BEL\FTK.K101-1.csv and .pickle - 

2020-09-09 15:48:19,638 []  ... saved output ..\data\instances\DNB-NR_FTK-2019-06_2019-12-31_MOD_FTK-BEL\FTK.K101-2.csv and .pickle - 

2020-09-09 15:48:22,607 []  ... saved output ..\data\instances\DNB-NR_FTK-2019-06_2019-12-31_MOD_FTK-BEL\FTK.K201-1.csv and .pickle - 

2020-09-09 15:48:23,273 []  ... saved output ..\data\instances\DNB-NR_FTK-2019-06_2019-12-31_MOD_FTK-BEL\FTK.K201-2.csv and .pickle - 

2020-09-09 15:48:24,173 []  ... saved output ..\data\instances\DNB-NR_FTK-

In [11]:
# create html files
# for large tables this will take a lot of time!

for table in tables:
    ViewFileRenderedGrid.viewRenderedGrid(xbrl_instance,
             join(XBRL_INSTANCES_PATH, subdir, table.genLabel(lang = LANGUAGE,strip = True, role = euRCcode)+".html"),
             viewTblELR = table,
             lang = LANGUAGE, 
             sourceView = None,
             diffToFile = False)

2020-09-09 15:48:35,812 [info] Saved output HTML to ..\data\instances\DNB-NR_FTK-2019-06_2019-12-31_MOD_FTK-BEL\K000.html - DNB-NR_FTK-2019-06_2019-12-31_MOD_FTK-BEL.XBRL 

2020-09-09 15:48:35,852 [info] Saved output HTML to ..\data\instances\DNB-NR_FTK-2019-06_2019-12-31_MOD_FTK-BEL\K050.html - DNB-NR_FTK-2019-06_2019-12-31_MOD_FTK-BEL.XBRL 

2020-09-09 15:48:36,026 [info] Saved output HTML to ..\data\instances\DNB-NR_FTK-2019-06_2019-12-31_MOD_FTK-BEL\K101-1.html - DNB-NR_FTK-2019-06_2019-12-31_MOD_FTK-BEL.XBRL 

2020-09-09 15:48:36,285 [info] Saved output HTML to ..\data\instances\DNB-NR_FTK-2019-06_2019-12-31_MOD_FTK-BEL\K101-2.html - DNB-NR_FTK-2019-06_2019-12-31_MOD_FTK-BEL.XBRL 

2020-09-09 15:48:38,325 [info] Saved output HTML to ..\data\instances\DNB-NR_FTK-2019-06_2019-12-31_MOD_FTK-BEL\K201-1.html - DNB-NR_FTK-2019-06_2019-12-31_MOD_FTK-BEL.XBRL 

2020-09-09 15:48:38,425 [info] Saved output HTML to ..\data\instances\DNB-NR_FTK-2019-06_2019-12-31_MOD_FTK-BEL\K201-2.html - DNB

## Example to read a template from the pickle files

The easiest way to access the data of a separate template is to read the corresponding pickle-file.

In [12]:
t = tables[10].genLabel(lang = LANGUAGE,strip = True, role = euRCcode)
df = pd.read_pickle(join(XBRL_INSTANCES_PATH, subdir, t + ".pickle"))
df

FileNotFoundError: [Errno 2] No such file or directory: '..\\data\\instances\\DNB-NR_FTK-2019-06_2019-12-31_MOD_FTK-BEL\\K206.pickle'

## Validate instance from Arelle

It should be possible to validate the instance (performing the validation rules within the taxonomy) with Arelle with the following code. But we did not test this!

In [12]:
# from arelle import Validate
# Validate.validate(xbrl_instance)